## **Environment Setup**
Run all when initiating session

In [ ]:
# retrieve config file, if using colab (nothing should happen if you use Windows)
!cp './drive/My Drive/Live Workspace/generative-facial-cosmetics/encoder_engineering/config.py' '.'
!mkdir data
!cp -a './drive/My Drive/Live Workspace/generative-facial-cosmetics/encoder_engineering/data/' '.'
!mkdir models
!cp -a './drive/My Drive/Live Workspace/generative-facial-cosmetics/encoder_engineering/models/' '.'
!mkdir technical
!cp -a './drive/My Drive/Live Workspace/generative-facial-cosmetics/encoder_engineering/technical/' '.'
!mkdir utils
!cp -a './drive/My Drive/Live Workspace/generative-facial-cosmetics/encoder_engineering/utils/' '.'

In [ ]:
from config import *                            # config.py
from matplotlib import pyplot as plt 
import os
%matplotlib inline

## **GCS Integration**
Run all when initiating session

In [ ]:
if isWindows():
  import utils.gcs_windows as gcs 
elif isColab():
  import utils.gcs_colab as gcs
else:
  raise NotImplementedError('OS is not supported yet')

if isColab():
  gcs.init()
# no need of buckets for this project

## **Model Environment Setup**

In [ ]:
### tf-side debug ###
# more info: https://github.com/tensorflow/tensorflow/issues/29931
import tensorflow as tf
temp = tf.zeros([2, 16, 16, 3])  # Or tf.zeros
tf.keras.applications.vgg16.preprocess_input(temp)
print('')

In [ ]:
from data.pipeline import *
from technical.accelerators import strategy
from utils.generator_loading_utils import load_generator_checkpoint
from utils.encoder_loading_utils import load_encoder_checkpoint
from utils.face_utils import detect_and_crop_lips, replace_lips

In [ ]:
##### Create and load saved generator here (checkpoint) #####
# please put generator's saved checkpoint in OUTPUT/generator_checkpoints (GCS for colab)
RUN = True
if RUN:
  generator = load_generator_checkpoint(strategy, model_type='256')
  print(generator)
else:
  print("Running switch for this cell is off, skipping")

In [ ]:
##### Create and load saved encoder here (checkpoint) #####
# please put encoder's saved checkpoint in OUTPUT/encoder_checkpoints (GCS for colab)
RUN = True
if RUN:
  encoder = load_encoder_checkpoint(strategy)
  print(encoder)
else:
  print("Running switch for this cell is off, skipping")

In [ ]:
LATENT_SIZE = encoder.model.layers[-1].output.shape[1:] 
IMAGE_SHAPE = encoder.model.layers[0].output.shape[1:] 
IMAGE_SIZE = [IMAGE_SHAPE[0], IMAGE_SHAPE[0]]
print("Latent size: " + str(LATENT_SIZE))
print("Image shape: " + str(IMAGE_SHAPE))

## **Testing**

In [ ]:
import numpy as np
import os

In [ ]:
## input
img_name = 'test_2.png'
img_path = os.path.join(DIR, os.path.join('samples', img_name))
img = load_image(img_path)
plt.imshow(img[:, :, [2, 1, 0]])

In [ ]:
## crop
cropped_lips, p_data, img = detect_and_crop_lips(img_full=img)
cropped_lips = np.array(cropped_lips)/255
#plt.imshow(cropped_lips)

In [ ]:
resized = cropped_lips #cv2.resize(cropped_lips, dsize=(90, 90), interpolation=cv2.INTER_CUBIC)
plt.imshow(resized)

In [ ]:
## Encode the cropped lips
encoded = encoder.model(np.expand_dims(resized, 0))
#print(encoded)

In [ ]:
## Regeneration (w/o injections)
regenerated = generator.model(encoded, training=False)[0]
plt.imshow(regenerated)
print("Visual loss: " + str(float(tf.keras.losses.MSE(tf.keras.backend.flatten(resized), tf.keras.backend.flatten(regenerated)))))

In [ ]:
## Replacement
#img = cv2.resize(img, dsize=(90, 90), interpolation=cv2.INTER_CUBIC)
new_img = replace_lips(regenerated.numpy(), p_data, img_full=img)

In [ ]:
injectible = generator.get_injectible_model(strategy)
inputs = injectible.inputs
for layer in inputs:
  print(layer.name)

In [ ]:
## Load to-be-injected image
img2_name = 'purple_lips.png'
img2_path = os.path.join(DIR, os.path.join('samples', img2_name))
img2 = load_image(img2_path)
plt.imshow(img2[:, :, [2, 1, 0]])

In [ ]:
## Crop to-be-injected image
cropped_lips2, p_data2, img2 = detect_and_crop_lips(img_full=img2)
cropped_lips2 = np.array(cropped_lips2)/255
plt.imshow(cropped_lips2)

In [ ]:
## Encode the cropped lips
encoded2 = encoder.model(np.expand_dims(cropped_lips2, 0))
#print(encoded2)

In [ ]:
## Regeneration (w/ injections)
regenerated_mix = injectible([encoded]*6 + [encoded2*2], training=False)[0]
plt.imshow(regenerated_mix)

In [ ]:
## Replacement
new_img = replace_lips(regenerated_mix.numpy(), p_data, img_full=img)

In [ ]:
plt.imshow(cv2.imdecode(np.fromstring(cv2.imencode(".png", img)[1].tostring(), np.uint8), cv2.IMREAD_COLOR)[:, :, [2, 1, 0]])

In [ ]:
## Regeneration (w/o injections)
regenerated = injectible([encoded2]*6 + [encoded2*2], training=False)[0]
plt.imshow(regenerated)
print("Visual loss: " + str(float(tf.keras.losses.MSE(tf.keras.backend.flatten(cropped_lips2), tf.keras.backend.flatten(regenerated)))))

## **Experiments**
Just a playground for trying out codes, nothing related at all, do not execute

In [ ]:
img_pil = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGBA))
plt.imshow(img_pil)

In [ ]:
min_x, min_y, max_x, max_y, offset_x, offset_y = p_data
cropped_lips_pil = Image.fromarray((cropped_lips*255).astype('uint8'))
cropped_lips_pil = cropped_lips_pil.crop((offset_x, offset_y, offset_x+(max_x-min_x), offset_y+(max_y-min_y)))
img_pil = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGBA))
img_pil.paste(cropped_lips_pil, (min_x, min_y), cropped_lips_pil)
plt.imshow(img_pil)

In [ ]:
min_x, min_y, max_x, max_y, offset_x, offset_y = p_data
img_tmp = (cropped_lips*255).astype('uint8')
cropped_lips_pil = Image.fromarray(img_tmp)
cropped_lips_pil = cropped_lips_pil.crop((offset_x, offset_y, offset_x+(max_x-min_x), offset_y+(max_y-min_y)))
plt.imshow(cropped_lips_pil)

In [ ]:
## DEPRECATED encode cv2 image to str rep. of bytes
with open("encoded_tmp.txt", "w") as f:
    f.write(str(cv2.imencode(".png", img)[1].tostring()))

In [ ]:
## DEPRECATED retrieve the str rep. of bytes and convert back to cv2
with open("encoded_tmp.txt", "r") as f:
    fstr = f.read()
fbytes = eval(fstr)
plt.imshow(cv2.imdecode(np.fromstring(fbytes, np.uint8), cv2.IMREAD_COLOR)[:, :, [2, 1, 0]])

In [ ]:
## new method (more stable storage)
import base64
orig_bytes = cv2.imencode(".png", img)[1].tostring()
pre_transportable = base64.b64encode(orig_bytes)
transportable = str(pre_transportable, "ANSI")
post_transportable = bytes(transportable, "ANSI")
bbytes = base64.b64decode(post_transportable)
plt.imshow(cv2.imdecode(np.fromstring(bbytes, np.uint8), cv2.IMREAD_COLOR)[:, :, [2, 1, 0]])

In [ ]:
with open("encoded_tmp.txt", "w") as f:
    f.write(transportable)

In [ ]:
with open("encoded_tmp.txt", "r") as f:
    transportable = f.read()
post_transportable = bytes(transportable, "ANSI")
bbytes = base64.b64decode(post_transportable)
plt.imshow(cv2.imdecode(np.fromstring(bbytes, np.uint8), cv2.IMREAD_COLOR)[:, :, [2, 1, 0]])

In [ ]:
def AdaIN(x):
    # Normalize x[0] (image representation)
    mean = tf.keras.backend.mean(x[0], axis = [1, 2], keepdims = True)
    std = tf.keras.backend.std(x[0], axis = [1, 2], keepdims = True) + 1e-7
    y = (x[0] - mean) / std
    
    # Reshape scale and bias parameters
    pool_shape = [-1, 1, 1, y.shape[-1]]
    scale = tf.keras.backend.reshape(x[1], pool_shape)
    bias = tf.keras.backend.reshape(x[2], pool_shape)
    
    # Multiply by x[1] (GAMMA) and add x[2] (BETA)
    return y * scale + bias

tmp_model = tf.keras.models.load_model("./outputs/saved_models/current/generator-p_5-e_9999.h5", custom_objects={'AdaIN': AdaIN})

In [ ]:
## Regeneration (w/o injections)
regenerated = tmp_model(encoded, training=False)[0]
plt.imshow(regenerated)

In [ ]:
orig_bytes = cv2.imencode(".png", np.array(new_img))[1].tostring()
pre_transportable = base64.b64encode(orig_bytes)
transportable = str(pre_transportable, "ANSI")